In [ ]:
%load_ext autoreload
%autoreload 2
import lenstronomy
import numpy as np
import h5py
import matplotlib.pyplot as plt
import pandas as pd

# lenstronomy stuff
from astropy.cosmology import FlatLambdaCDM
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.PointSource.point_source import PointSource
from lenstronomy.Plots import lens_plot
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver


- need to start with defining the ground truth? 
- you can have ground truth lens parameters, ground truth H_0, 
- a table of ground truth params, predicted params, etc?

1. define ground truth lens params (either from a catalog or generate randomly)
2. construct emulated posteriors for those lens params
3. define ground truth H-0
4. compute ground truth time delays
5. constructed emulated time delay measurement
6. construct Ddt posteriors from lens param posterior + time delay measurement

In [ ]:
y_pred_doppel_seq_list = np.load('/Users/smericks/Desktop/StrongLensing/STRIDES14results/paper_results/doppelganger_test_set/results/y_pred_list_epoch10.npy')
std_pred_doppel_seq_list = np.load('/Users/smericks/Desktop/StrongLensing/STRIDES14results/paper_results/doppelganger_test_set/results/std_pred_list_epoch10.npy')

# load in predictions from broad training
file_prefix = '/Users/smericks/Desktop/StrongLensing/STRIDES14results/paper_results/broad_training/diag_no_R_src/J2145_rerun_take2/'
file_prefix = '/Users/smericks/Desktop/StrongLensing/STRIDES14results/paper_results/broad_training/diag_no_R_src/J2145_rerun_take2/'
file_path = file_prefix+'doppelganger_predictions.h5'
h5f = h5py.File(file_path, 'r')
y_test_doppel = h5f.get('y_test')[:]
h5f.close()

# ordering is 0x,1x,2x,4x
y_pred_doppel = y_pred_doppel_seq_list[2]
std_pred_doppel = std_pred_doppel_seq_list[2]

In [ ]:
import lens_sample

my_sample = lens_sample.LensSample(y_test_doppel,y_pred_doppel,std_pred_doppel,param_indices=[0,3,4,5,6,7,1,2,8,9])
my_sample.populate_image_positions()
my_sample.populate_fermat_differences()
my_sample.populate_redshifts()
# what if I populate the actual redshifts instead? That will change the ground 
# truth time delays & more accurately reflect the selection function
doppel_folders = ['ATLASJ2344-3056', 'J0029-3814', 'J2205-3727', 
    'SDSSJ1251+2935', 'WISEJ0259-1635', 'DESJ0405-3308','J1131-4419',
    'PSJ1606-2333', 'W2MJ1042+1641', 'DESJ0420-4037', 'J2145+6345', 
    'SDSSJ0248+1913', 'WG0214-2105']
for r in range(0,len(my_sample.lens_df)):
    metadata_path = ('/Users/smericks/Desktop/StrongLensing/deep-lens-modeling/'+
        'paper/doppelganger_test_set/doppel_images/'+doppel_folders[r]+'/metadata.csv')
    paltas_config = pd.read_csv(metadata_path)
    # lens redshift
    my_sample.lens_df.loc[r,'z_lens'] = paltas_config.loc[0,'main_deflector_parameters_z_lens']
    # source redshift
    my_sample.lens_df.loc[r,'z_src'] = paltas_config.loc[0,'source_parameters_z_source']

my_sample.populate_truth_Ddt()
my_sample.populate_truth_time_delays()
my_sample.populate_measured_time_delays()
my_sample.lens_df

In [ ]:
# let's try out fpd samps vs. truth

for j in range(0,13):

    pred_samples = my_sample.pred_fpd_samples(j)
    truth = my_sample.lens_df.iloc[j][['fpd01', 'fpd02', 'fpd03']]

    fig,axs = plt.subplots(1,3,figsize=(10,5))
    for i in range(0,3):
        axs[i].hist(pred_samples[i],label='predicted')
        axs[i].vlines(truth[i],0,200,color='red',label='truth')
        axs[i].set_xlabel('$\Delta\phi_{0%d}$'%(i+1))
        if i == 2:
            axs[i].legend()

    plt.suptitle('Doppelganger %d'%(j))
    plt.show()


In [ ]:
for i in range(0,13):
    Ddt_samps, Ddt_likelihoods = my_sample.Ddt_posterior(i,td_uncertainty=2)
    binwidth = 100
    plt.hist(Ddt_samps,weights=Ddt_likelihoods,label='Ddt posterior',bins=np.arange(0,15000+binwidth,binwidth))
    plt.vlines(my_sample.lens_df.loc[i,'Ddt_Mpc_truth'],0,0.03,color='red',label='truth')
    plt.title('Ddt Doppelganger %d'%(i))
    plt.legend()
    plt.show()

In [ ]:
for i in range(0,13):
    h0_samps, h0_weights = my_sample.H0_individual_lens(i)
    binwidth = 1
    n,_,_ = plt.hist(h0_samps,weights=h0_weights,label='H0 posterior',bins=np.arange(0,150+binwidth,binwidth))
    plt.vlines(70.,0,np.max(n),color='red',label='truth')
    plt.title('H0 Doppelganger %d'%(i))
    plt.legend()
    plt.show()

Now, let's do one joint constraint on H0 with all the lenses

In [ ]:
h0_samps, h0_weights = my_sample.H0_joint_inference()

In [ ]:
import inference_utils
from plot_utils import PALETTE

plt.figure(dpi=300)
binwidth = 1
n,_,_ = plt.hist(h0_samps,weights=h0_weights,label='Posterior',
    bins=np.arange(0,150+binwidth,binwidth),color=PALETTE['medium_purple'])
plt.vlines(70.,0,np.max(n),color='black',label='Truth',linestyle='--')
plt.title('13 Doppelgangers')
plt.legend(fontsize=13)
plt.xlabel('$H_0$ (km/s/Mpc)',fontsize=13)
plt.ylabel('Probability Density',fontsize=13)
h0_mean,h0_sigma = inference_utils.median_sigma_from_samples(h0_samps,h0_weights)
plt.text(80,0.038,'$H_0$ = %.1f +/- %.1f'%(h0_mean,h0_sigma),color=PALETTE['medium_purple'],fontsize=13)

plt.show()

# TODO: 
- for time delays, let's assert 5 days error for 5 season measurement, 2 days for 10 season measurement